In [2]:
import os
import pandas as pd
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
import os
import re
import pandas as pd
import matplotlib.pyplot as plt
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build

In [3]:
masterDB = pd.read_parquet("Master_DB_df.parquet")
masterDB.shape

(31138, 86)

In [4]:
print(masterDB.columns.tolist())
masterDB[["ePOS Code", "PIC NAME 1 Contact Number","PIC 1 Source", "PIC NAME 2 Contact Number","PIC 2 Source", "PIC NAME 3 Contact Number","PIC 3 Source", "PIC Verified"]]

['ePOS Code', 'Company Code', 'Date', 'ACRA REGISTERED NAME', 'Brand/Deal Name/Business Name', 'Sub Domain Link (If Lead is already available in Backend) Fill only when EPOS client', 'Tele Sales or MR (For KPI - Internal)', 'Name of the Market Researcher', 'Original Source (Marketing)', 'Marketing Source (Do not fill anything if the leads are from Hubspot, EPOS clients)', 'Company Registration date / Date Established', 'Company_Age', 'Company_Age_Group', 'Company Registration Number (UEN)', 'Primary SSIC Code', 'Secondary SSIC Code', 'Hubspot ID (Company)', 'Hubspot ID(Deal)', 'Hubspot ID(Contact)', 'Website URL', 'Business Type', 'Facebook Page', 'Instagram URL', 'Linkedin URL', 'Tik Tok URL', 'Ownership Type', 'Parent Industry Type', 'Industry Type', 'Sub Industry', 'Business model', 'Presence of Multiple Outlets', 'Number of Outlets (Write in #)', 'Region', 'Planning Area', 'Business Location Type', 'Registered Address (Block & Street)', 'Registered Address (Unit #)', 'Registered Ad

,ePOS Code,PIC NAME 1 Contact Number,PIC 1 Source,PIC NAME 2 Contact Number,PIC 2 Source,PIC NAME 3 Contact Number,PIC 3 Source,PIC Verified
0,EPOS600000,"65 9664 6332, 65 6734 2782",,65 8324 9880,,65 9021 5459,,
1,EPOS599999,65 8089 1279,,,,"65 1234 4567\n[65 1243 6745],\n65 3421 7654 \n...",,
2,EPOS599998,65 9873 1816,,,,,,
3,EPOS599997,,,,,,,None
4,EPOS599996,65 9006 9893,,,,,,
...,...,...,...,...,...,...,...,...
31133,EPOS147259,65 6741 7288,Google,,,,,None
31134,EPOS147258,65 6481 7773,Facebook,,,,,None
31135,EPOS147257,65 9489 2283,Facebook,65 8775 1162,Facebook,,,None
31136,EPOS147256,65 8835 5766,Official Website,65 6844 5575,Official Website,,,None


In [ ]:
import sqlite3

# Create in-memory database
conn = sqlite3.connect(':memory:')

# Load df into SQL table
masterDB.to_sql('masterDB', conn, index=False, if_exists='replace')

# Run SQL queries
result = pd.read_sql_query("""
                           

    SELECT "ePOS Code", "Date of the 1st Call", "PIC NAME 1 Contact Number","PIC 1 Source", "PIC NAME 2 Contact Number","PIC 2 Source", "PIC NAME 3 Contact Number","PIC 3 Source", "PIC Verified"
    FROM masterDB 
    WHERE "PIC Verified" IS NOT NULL 
    AND "PIC Verified" != ''
    AND "PIC 1 Source" != ''
                           
                           
""", conn)

result

,ePOS Code,Date of the 1st Call,PIC NAME 1 Contact Number,PIC 1 Source,PIC NAME 2 Contact Number,PIC 2 Source,PIC NAME 3 Contact Number,PIC 3 Source,PIC Verified
0,EPOS297727,29/12/2025,65 9387 0743,Lusha,65 9768 2970,Lusha,65 6284 0349,Official Website,PIC 2
1,EPOS297581,07/01/2026,"65 6852 2859, 65 9782 4541",Lusha,,,"65 6852 2852, 65 9830 3430, 65 9012 0165, 65 9...",Official Website,PIC 1
2,EPOS297578,29/12/2025,"65 6828 0887, 65 8128 0814",Lusha,65 9277 3315,Lusha,65 6777 7183,Official Website,PIC 3
3,EPOS749708,05/08/2025,65 8511 0043,Lusha,65 9188 0459,,,,PIC 1
4,EPOS748889,27/01/2026,"65 9173 6407, 65 6249 1535",Lusha,65 6226 2933,,,,PIC 1
...,...,...,...,...,...,...,...,...,...
75,EPOS198646,29/01/2026,65 9617 8869,Official Website,65 6289 8369,Official Website,,,PIC 1
76,EPOS198645,29/01/2026,65 6344 5924,Google,,,,,PIC 1
77,EPOS198644,29/01/2026,65 9819 2416,Facebook,,,,,PIC 1
78,EPOS198643,29/01/2026,65 6295 1083,Google,,,,,PIC 1


In [10]:
# Load result into a new SQL table
result.to_sql('result_table', conn, index=False, if_exists='replace')

# Count verified sources
result2 = pd.read_sql_query("""
    SELECT 
        CASE 
            WHEN "PIC Verified" = 'PIC 1' THEN "PIC 1 Source"
            WHEN "PIC Verified" = 'PIC 2' THEN "PIC 2 Source"
            WHEN "PIC Verified" = 'PIC 3' THEN "PIC 3 Source"
        END AS verified_source,
        COUNT(*) AS count
    FROM result_table
    GROUP BY verified_source
    HAVING verified_source IS NOT NULL AND verified_source != ''
    ORDER BY count DESC
""", conn)

result2


,verified_source,count
0,Official Website,22
1,Lusha,20
2,Google,11
3,Facebook,7
4,Others,6
